In [1]:
from dotenv import load_dotenv

load_dotenv("../.envrc")

True

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, openai_embedding)

In [17]:
query = "대통령의 임기는?"
#유사 문서 검색
docs = db.similarity_search(query)
print(docs[0])

page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의' metadata={'total_pages': 14, 'moddate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'creationdate': '2024-04-01T21:26:24+09:00', 'page': 6, 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'page_label': '7'}


In [18]:
#유사 문서 검색 및 유사도 출력
db.similarity_search_with_score(query)

[(Document(metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'moddate': '2024-04-01T21:26:24+09:00', 'page_label': '7', 'creator': 'PyPDF', 'total_pages': 14, 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page': 6}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'),
  1.0360676050186157),
 (Document(metadata={'page': 7, 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'total_pages': 14, 'page_label': '8', 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'moddate': '2024-04-01T21:26:24+09:00', 'creation

In [19]:
from langchain_chroma import Chroma

Chroma().delete_collection()

In [21]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)


#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

#save to disk
db2 = Chroma.from_documents(docs, ko_embedding, persist_directory="./chroma_db")

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=ko_embedding)

query = "대통령의 임기는?"
result = db3.similarity_search(query)
print(result)
print(result[0].page_content)

[Document(id='cbc6014d-c781-4b79-a392-3000af195f6f', metadata={'total_pages': 14, 'page': 6, 'creationdate': '2024-04-01T21:26:24+09:00', 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page_label': '7'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'), Document(id='74e6e124-67f7-4473-8981-723cc833307f', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'page_label': '7', 'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'creator': 'PyPDF', 'moddate': '2024-04

In [22]:
import chromadb
#collection을 저장할 경로 지정
client = chromadb.PersistentClient(path="collection_example")
#client가 잘 연결되어 있는지 확인
client.heartbeat()

1749731864455559000

In [23]:
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
#OpenAI 임베딩 모델 활용
embedding_function = openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')
#Huggingface 오픈소스 임베딩 모델 활용
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="jhgan/ko-sbert-nli")

collection = client.create_collection(name="korean_law", embedding_function=embedding_function)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader


# load the document and split it into chunks
loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

collection.add(
    ids = [str(i) for i in range(len(docs))],
    documents=[i.page_content for i in docs],
    metadatas=[i.metadata for i in docs]
)

In [25]:
#name에 collection 이름, embedding_function은 collection 저장 시 지정한 임베딩 모델 지정
collection = client.get_collection(name="korean_law", embedding_function=embedding_function)
collection

Collection(name=korean_law)

In [26]:
#1페이지에서 직업 선택의 자유와 유사한 청크 3개 검색
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": 1},
)

{'ids': [['7', '5', '4']],
 'embeddings': None,
 'documents': [['제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.',
   '제12조 ①모든 국민은 신체의 자유를 가진다. 누구든지 법률에 의하지 아니하고는 체포ㆍ구속ㆍ압수ㆍ수색 또는 심문\n을 받지 아니하며, 법률과 적법한 절차에 의하지 아니하고는 처벌ㆍ보안처분 또는 강제노역을 받지 아니한다.\n②모든 국민은 고문을 받지 아니하며, 형사상 자기에게 불리한 진술을 강요당하지 아니한다.\n③체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시\n하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피 또는 증거인멸의 염려가\n있을 때에는 사후에 영장을 청구할 수 있다.\n④누구든지 체포 또는 구속을 당한 때에는 즉시 변호인의 조력을 받을 권리를 가진다. 다만, 형사피고인이 스스로\n변호인을 구할 수 없을 때에는 법률이 정하는 바에 의하여 국가가 변호인을 붙인다.\n⑤누구든지 체포 또는 구속의 이유와 변호인의 조력을 받을 권리가 있음을 고지받지 아니하고는 체포 또는 구속을',
   '법제처                                                            2                                                       국가법령정보센터\n대한민국헌법\n \n 

In [27]:
#5페이지 이후의 청크 중에서 직업 선택의 자유와 관련한 문서 3개 검색
# $eq - 일치 (string, int, float)
# $ne - 불일치 (string, int, float)
# $gt - 초과 (int, float)
# $gte - 이상 (int, float)
# $lt - 미만 (int, float)
# $lte - 이하 (int, float)
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": {"$gte": 5}}
)

{'ids': [['47', '45', '24']],
 'embeddings': None,
 'documents': [['법제처                                                            13                                                       국가법령정보센터\n대한민국헌법\n⑤국가는 농ㆍ어민과 중소기업의 자조조직을 육성하여야 하며, 그 자율적 활동과 발전을 보장한다.\n \n제124조 국가는 건전한 소비행위를 계도하고 생산품의 품질향상을 촉구하기 위한 소비자보호운동을 법률이 정하는 바\n에 의하여 보장한다.\n \n제125조 국가는 대외무역을 육성하며, 이를 규제ㆍ조정할 수 있다.\n \n제126조 국방상 또는 국민경제상 긴절한 필요로 인하여 법률이 정하는 경우를 제외하고는, 사영기업을 국유 또는 공유\n로 이전하거나 그 경영을 통제 또는 관리할 수 없다.\n \n제127조 ①국가는 과학기술의 혁신과 정보 및 인력의 개발을 통하여 국민경제의 발전에 노력하여야 한다.\n②국가는 국가표준제도를 확립한다.',
   '지하며, 경제주체간의 조화를 통한 경제의 민주화를 위하여 경제에 관한 규제와 조정을 할 수 있다.\n \n제120조 ①광물 기타 중요한 지하자원ㆍ수산자원ㆍ수력과 경제상 이용할 수 있는 자연력은 법률이 정하는 바에 의하\n여 일정한 기간 그 채취ㆍ개발 또는 이용을 특허할 수 있다.\n②국토와 자원은 국가의 보호를 받으며, 국가는 그 균형있는 개발과 이용을 위하여 필요한 계획을 수립한다.\n \n제121조 ①국가는 농지에 관하여 경자유전의 원칙이 달성될 수 있도록 노력하여야 하며, 농지의 소작제도는 금지된다.\n②농업생산성의 제고와 농지의 합리적인 이용을 위하거나 불가피한 사정으로 발생하는 농지의 임대차와 위탁경영\n은 법률이 정하는 바에 의하여 인정된다.\n \n제122조 국가는 국민 모두의 생산 및 생활의 기반이 되는 국토의 효율적이고 균형있는 이용ㆍ개발과 보전

In [28]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": 1},
    where_document={"$contains": "직업"}
)

{'ids': [['7']],
 'embeddings': None,
 'documents': [['제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'moddate': '2024-04-01T21:26:24+09:00',
    'creator': 'PyPDF',
    'source': '../data/대한민국헌법(헌법)(제00010호)(19880225).pdf',
    'page': 1,
    'creationdate': '2024-04-01T21:26:24+09:00',
    'page_label': '2',
    'total_pages': 14,
    'producer': 'iText 2.1.7 by 1T3XT'}]],
 'distances': [[395.4745178222656]]}